# Verify AWS credentials propagate to cluster workers

NOTE: This should only really be necessary in a cloud compute environment where cluster 
worker threads/nodes are spawned via kubernetes.  Clusters on our HPC hardware will not
need to specfically send credentials to workers. 

In [1]:
import os
import logging

def myAWS_Credentials():
    """Test function to return AWS credential information."""
    return {
    "AWS_PROFILE": os.environ.get("AWS_PROFILE", "<not set>"),
    "AWS_ACCESS_KEY_ID": os.environ.get('AWS_ACCESS_KEY_ID', '<not set>'),
    "AWS_S3_ENDPOINT": os.environ.get('AWS_S3_ENDPOINT', '<not set>')    
}

##### V V V V V V V V V V V V V V V V ##### set the profile

os.environ['AWS_PROFILE'] = "example"  

##### ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ 


1) Establish AWS credentials.

In [2]:
%run ./Help_AWS_Credentials.ipynb
## Sets AWS_* secrets based on contents of $HOME/.aws/credentials

2. What credentials are set in the execution environment?

In [3]:
myAWS_Credentials()  ## should return the key for the above-named profile

{'AWS_PROFILE': 'example', 'AWS_ACCESS_KEY_ID': 'ABC123'}

3. Start a default cluster

In [4]:
## pick the correct helper for the environment where you are running...
%run ./Start_Dask_Cluster_Nebari.ipynb
# %run ./Start_Dask_Cluster_Nebari.ipynb
# %run ./Start_Dask_Cluster_Denali.ipynb
# %run ./Start_Dask_Cluster_Tallgrass.ipynb
# %run ./Start_Dask_Cluster_PangeoCHS.ipynb


The 'cluster' object can be used to adjust cluster behavior.  i.e. 'cluster.adapt(minimum=10)'
The 'client' object can be used to directly interact with the cluster.  i.e. 'client.submit(func)' 
The link to view the client dashboard is:
>  https://nebari.esipfed.org/gateway/clusters/dev.3f52ec52b3d2494bb008e44591f2b4d2/status


4. Force a worker to report the credentials on their node

In [5]:
client.submit(myAWS_Credentials).result() 
## This runs the myAWS_Credentials function on a remote node worker (not in this notebook).

{'AWS_PROFILE': '<not set>', 'AWS_ACCESS_KEY_ID': 'ABC123'}

5. Shut down client and cluster resources.

In [6]:
client.close(); cluster.close()

/home/conda/users/41ef378107ff90fc7fde32dde43db5e4e6e9bb184d9d537cc60311ec1b3f3d26-20230223-142312-805812-99-pangeo/lib/python3.9/site-packages/dask_gateway/client.py:1014: RuntimeWarning: coroutine 'rpc.close_rpc' was never awaited
  self.scheduler_comm.close_rpc()
